In [1]:
import numpy as np
import itertools

In [2]:
timesteps = list(range(3))

actions = list(itertools.combinations_with_replacement(timesteps,2))
print(actions)

In [3]:
def compute_utilities(strategy_1, strategy_2, n_players, n_concerts, n_lines):

    utilities = np.zeros(n_players)
    strategies = [strategy_1, strategy_2]

    positions_which_concert = np.zeros(n_players)
    positions_which_line = np.ones(n_players, dtype= float) * np.mean(sum(range(1,n_lines + 1))) # assume mean line value for every player in first concert
    time_spent_at_current_stage = np.zeros(n_players)

    # iterate through time
    for time in range(3):
        
        # update positions
        for player in range(n_players):

            player_strategy = strategies[player]
            
            # check if player will move
            moved = 0
            for i in range(len(player_strategy)):

                if (player_strategy[i] == time):
                    positions_which_concert[player] = i + 1 # eg player_strategy[i] = 0 means go to concert 1 at time 0. 
                    time_spent_at_current_stage[player] = 0
                    moved = 1

            # increase time at current stage
            if(not moved):
                time_spent_at_current_stage[player] = time_spent_at_current_stage[player] + 1

        # check which lines each player is at
        for concert in range(n_concerts):
            players = list(np.where(positions_which_concert == concert)[0])
           
            if(len(players) > 0):

                count_players = 0
           
                for time_spent in range(int(max(time_spent_at_current_stage[players])),-1, -1):
                    players_who_spent_this_time = list(np.where(time_spent_at_current_stage == time_spent)[0])
                    players_correct_time_and_stage = list(np.intersect1d(players, players_who_spent_this_time))
                    #print(players_correct_time_and_stage)

                    n_players_same_time = len(players_correct_time_and_stage)
                    
                    if (n_players_same_time > 0):
                        # assuming two players
                        if (n_players_same_time == 1 and count_players == 0):
                            positions_which_line[players_correct_time_and_stage] = 5
                            count_players += 1
                        elif (n_players_same_time == 1 and count_players == 1):
                            positions_which_line[players_correct_time_and_stage] = 1
                        else:
                            positions_which_line[players_correct_time_and_stage] = 3
    
        # check current concert and update utilities for each player
        for player in range(n_players):
            if (positions_which_concert[player] == time):
                if(time == 0):
                    utilities[player] += 0
                else:
                    utilities[player] += 1*positions_which_line[player]
            else:
                utilities[player] -= 2

    return utilities


In [4]:
n_concerts = 3
n_lines = 2
n_players = 2

n_actions = len(actions)

utilities_normal_form_P1 = np.zeros((n_actions, n_actions))
utilities_normal_form_P2 = np.zeros((n_actions, n_actions))

for i in range(n_actions):
    for j in range(n_actions):
        utilities = compute_utilities(actions[i], actions[j], n_players, n_concerts, n_lines)
        utilities_normal_form_P1[i][j] = utilities[0]
        utilities_normal_form_P2[i][j] = utilities[1]

print("Normal form P1")
print(utilities_normal_form_P1)
print()
print("Normal form P2")
print(utilities_normal_form_P2)


In [ ]:
import nashpy as nash

game = nash.Game(utilities_normal_form_P1, utilities_normal_form_P2)
equilibria = game.support_enumeration()
for eq in equilibria:
    print(eq)

print(utilities_normal_form_P1)
print(equilibria)
